In [1]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gzip
import re

# Hong's job description

Attached please find the venn diagrams we generated based on the lists your provided, I also sent you the shared enhanced binding gene lists for both FC>1.2 (333 genes) and FC>1.5 (87 genes). Please generate TSS centered heatmaps for these two list genes respectively. Also include Y78A in these heatmaps. The dark blue color we usually use for Flag ChIP peaks would be good.

At the meantime, you can help to generate venn diagram for WT, T1, T2 and T3, and the heatmaps for co-up peaks.

For the motif analysis, please analysis the following four groups of genes: (1) 333 genes with FC>1.2 enhanced binding; (2) 87 genes with FC>1.5 enhanced binding; (3) T3 vs.WT FC>1.5 enhanced binding. For this three, please find enriched motifs in TSS +/- 2kb region. And then (4) look for enriched motifs in wild type ENL binding peaks.

Comments 2018-8-17
Tss will change to the peak center
venn diagram for all Ts and ENL

# 2. Generate the venn diagram for peaks for T1 T2 T3

In [2]:
mkdir -p ./4.venn_diagram

In [3]:
workPath = '/home/jlyu/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/Run_On_Lvjie/4.venn_diagram/'
os.chdir(workPath)
!pwd

/home/jlyu/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/Run_On_Lvjie/4.venn_diagram


In [4]:
exp = 'HK2'
bed_path = '/mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/'
bedFiles = glob.glob(f'{bed_path}{exp}*nsp_peaks.bed')
# extract the first three columns of the bed files
for bedFile in bedFiles:
    outFile = bedFile.split('/')[-1].split('.')[0]
    line = "awk \'BEGIN{OFS=\"\\t\"}{print $1,$2,$3}\'" + f' {bedFile} > {outFile}.bed'
    print(line)
    os.system(line)

awk 'BEGIN{OFS="\t"}{print $1,$2,$3}' /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T2.nsp_peaks.bed > HK2_F_T2.bed
awk 'BEGIN{OFS="\t"}{print $1,$2,$3}' /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_Y78A.nsp_peaks.bed > HK2_F_Y78A.bed
awk 'BEGIN{OFS="\t"}{print $1,$2,$3}' /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T1.nsp_peaks.bed > HK2_F_T1.bed
awk 'BEGIN{OFS="\t"}{print $1,$2,$3}' /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_T3.nsp_peaks.bed > HK2_F_T3.bed
awk 'BEGIN{OFS="\t"}{print $1,$2,$3}' /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/HK2_F_ENL.nsp_peaks.bed > HK2_F_ENL.bed


In [7]:
nodes_ppn = [1, 1]
memory = 4
wait_time = [20, 00]
quene = "short"
files = glob.glob('*.bed')
files.sort()
files = files[:-1]
names = [ifile.split('/')[-1].split('.')[0] for ifile in files]

In [8]:
# open the initialization header for pbs and read the contents
with open('/home/jlyu/xc3/experiment/initial.pbs','r') as f:
    pbs_header = f.readlines()

# write each sample a pbs files
experiment_name = "{}.ato".format('venn_diagram')
with open(experiment_name,'w') as f:
    pbs_initial = pbs_header[:]
    # configuration for the experiments
    # pbs_initial[1]::job name
    pbs_initial[1] = pbs_initial[1].format('venn')
    # pbs_initial[2]::nodes and ppn
    pbs_initial[2] = pbs_initial[2].format(*nodes_ppn)
    # pbs_initial[4]::memeroy
    pbs_initial[4] = pbs_initial[4].format(memory)
    # pbs_initial[5]::waiting time
    pbs_initial[5] = pbs_initial[5].format(*wait_time)
    # pbs_initial[11]::err output
    pbs_initial[11] = pbs_initial[11].format('venn')
    # pbs_initial[12]::log
    pbs_initial[12] = pbs_initial[12].format('venn')
    # pbs_initial[14]::quene
    pbs_initial[14] = pbs_initial[14].format(quene)
    
    #write all the configurations into the pbs file
    for line in pbs_initial:
        f.write(line)
    line = f'cd {workPath}'
    f.write(line + '\n')

    treat_files = files
    print(treat_files)
    treat_files_format = '{} ' * len(treat_files)
    treat_files_format = treat_files_format.format(*treat_files)
    output_venn = '{}_all.venn.pdf'.format(exp)
    output_bed = '{}_all.venn.bed'.format(exp)
    line = 'python /home/jlyu/xc3/software/bed_overlap4.py -v {} -i {} {}'.format(output_venn, output_bed, treat_files_format)
    print(line)
    f.write(line + '\n')    

['HK2_F_ENL.bed', 'HK2_F_T1.bed', 'HK2_F_T2.bed', 'HK2_F_T3.bed']
python /home/jlyu/xc3/software/bed_overlap4.py -v HK2_all.venn.pdf -i HK2_all.venn.bed HK2_F_ENL.bed HK2_F_T1.bed HK2_F_T2.bed HK2_F_T3.bed 


# 3. Heatmap for subgroup coup peaks

The coup peaks comes from the venn analysis intersection from bedoverlap

In [2]:
!pwd

/home/jlyu/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/Run_On_Lvjie


In [10]:
!mkdir -p /home/jlyu/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/Run_On_Lvjie/4.coup_peaks

In [3]:
workPath = '/home/jlyu/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/Run_On_Lvjie/4.coup_peaks'
os.chdir(workPath)
!pwd

/home/jlyu/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/Run_On_Lvjie/4.coup_peaks


## 3.1 extract the 1.2 / 1.5 peaks as bed files then use bedoverlap to get the intersect

In [5]:
ths = [1.2, 1.5]
names = ['1_2','1_5']

In [4]:
!ls

1_2.Ts.ENL.peak.gz	  HK2.all.1_5.sig.anno.txt
1_2.Ts.ENL.peak.gz.txt	  HK2.all.1_5.str.bed
1_2.heatmap.peak.mat	  HK2_T1_WT.1_2.bed
1_5.Ts.ENL.peak.gz	  HK2_T1_WT.1_5.bed
1_5.Ts.ENL.peak.gz.txt	  HK2_T1_WT.merge.counts.anno.pval.txt
1_5.heatmap.peak.mat	  HK2_T2_WT.1_2.bed
293.all.1_2.new.heat.bed  HK2_T2_WT.1_5.bed
293.all.1_5.new.heat.bed  HK2_T2_WT.merge.counts.anno.pval.txt
HK2.1_2.peak.pdf	  HK2_T3_WT.1_2.bed
HK2.1_2.peak.sig.txt	  HK2_T3_WT.1_5.bed
HK2.1_5.peak.pdf	  HK2_T3_WT.merge.counts.anno.pval.txt
HK2.1_5.peak.sig.txt	  PeakMatrix_HK2_F_ENL_co-UP_genes.1_5.gz
HK2.all.1_2.bed		  PeakMatrix_HK2_F_ENL_co-UP_genes.gz
HK2.all.1_2.counts.txt	  PeakMatrix_HK2_F_T1_co-UP_genes.1_5.gz
HK2.all.1_2.heat.bed	  PeakMatrix_HK2_F_T1_co-UP_genes.gz
HK2.all.1_2.new.heat.bed  PeakMatrix_HK2_F_T2_co-UP_genes.1_5.gz
HK2.all.1_2.sig.anno.txt  PeakMatrix_HK2_F_T2_co-UP_genes.gz
HK2.all.1_2.str.bed	  PeakMatrix_HK2_F_T3_co-UP_genes.1_5.gz
HK2.all.1_5.bed		  PeakMatrix_HK2_F_T3_co-UP_genes.gz
HK2.

In [20]:
peakFiles = glob.glob('*.pval.txt')
peakFiles.sort()
for th, name in zip(ths, names):
    for peakFile in peakFiles:
        peakDf = pd.read_csv(peakFile, sep='\t', header=None, skiprows=[0])
        subId = peakDf.iloc[:,4].div(peakDf.iloc[:,5]) >= th
        print(peakDf[subId].shape,'-->', peakFile, f'{th}')
        outFile = peakFile.split(".")[0] + f'.{name}.bed'
        print(outFile)
        peakDf[subId].iloc[:, [0,1,2]].to_csv(f'{outFile}', sep='\t', header=None, index=False)

(640, 12) --> HK2_T1_WT.merge.counts.anno.pval.txt 1.2
HK2_T1_WT.1_2.bed
(940, 12) --> HK2_T2_WT.merge.counts.anno.pval.txt 1.2
HK2_T2_WT.1_2.bed
(9607, 12) --> HK2_T3_WT.merge.counts.anno.pval.txt 1.2
HK2_T3_WT.1_2.bed
(236, 12) --> HK2_T1_WT.merge.counts.anno.pval.txt 1.5
HK2_T1_WT.1_5.bed
(174, 12) --> HK2_T2_WT.merge.counts.anno.pval.txt 1.5
HK2_T2_WT.1_5.bed
(2746, 12) --> HK2_T3_WT.merge.counts.anno.pval.txt 1.5
HK2_T3_WT.1_5.bed


## 3.2 find the T1 T2 T3 overlapped peaks


# login a new node and copy the bash command 
```bash
module load epd/7.3-2                                                     
module load python-packages/2.7                                           
module load R/2.15.3                                                      
module load bowtie/1.1.0                                                  
module load tophat/2.1.0                                                  
#module load samtools/1.6                                                 
module load samtools/0.1.19                                               
export PYTHONPATH=/home/jlyu/xc3/software/python/site-packages:$PYTHONPATH

```
```bash
python /home/jlyu/xc3/software/bed_overlap3.py -v HK2.all.1_2.venn.pdf -i HK2.all.1_2.bed HK2_T1_WT.1_2.bed HK2_T2_WT.1_2.bed HK2_T3_WT.1_2.bed
python /home/jlyu/xc3/software/bed_overlap3.py -v HK2.all.1_5.venn.pdf -i HK2.all.1_5.bed HK2_T1_WT.1_5.bed HK2_T2_WT.1_5.bed HK2_T3_WT.1_5.bed
```

In [25]:
for i, name in enumerate(names):
    bedFiles = [glob.glob(f'*{treat}*{name}.bed')[0] for treat in ['T1','T2','T3']]
    outBed = f'{exp}.all.{name}.bed'
    outVenn = f'{exp}.all.{name}.venn.pdf'
    bed_files_format = ' '.join(bedFiles)
    line = 'python /home/jlyu/xc3/software/bed_overlap3.py -v {} -i {} {}'.format(outVenn, outBed, bed_files_format)
    print(line)
    

python /home/jlyu/xc3/software/bed_overlap3.py -v HK2.all.1_2.venn.pdf -i HK2.all.1_2.bed HK2_T1_WT.1_2.bed HK2_T2_WT.1_2.bed HK2_T3_WT.1_2.bed
python /home/jlyu/xc3/software/bed_overlap3.py -v HK2.all.1_5.venn.pdf -i HK2.all.1_5.bed HK2_T1_WT.1_5.bed HK2_T2_WT.1_5.bed HK2_T3_WT.1_5.bed


In [27]:
!ls

HK2.all.1_2.bed			      HK2_T2_WT.1_5.bed
HK2.all.1_5.bed			      HK2_T2_WT.merge.counts.anno.pval.txt
HK2_T1_WT.1_2.bed		      HK2_T3_WT.1_2.bed
HK2_T1_WT.1_5.bed		      HK2_T3_WT.1_5.bed
HK2_T1_WT.merge.counts.anno.pval.txt  HK2_T3_WT.merge.counts.anno.pval.txt
HK2_T2_WT.1_2.bed


## 3.3 Redraw the venn diagram for the shared 1.2 1.5 peaks
the results depends hong whether like it or not (TODO)

## 3.4 Extract the signal for heatmap
using the previous above 

In [28]:
#format the bed file into the bed file add three columns more; 
for name in names:
    Bed = f'{exp}.all.{name}.bed'
    outBed = f'{exp}.all.{name}.heat.bed'
    line = "awk 'BEGIN{OFS=\"\\t\"}{if (NR>1){print $1,$2,$3,\"none\",\".\",\"+\"}}'" + f' {Bed} > {outBed}'
    print(line)
    os.system(line)

awk 'BEGIN{OFS="\t"}{if (NR>1){print $1,$2,$3,"none",".","+"}}' HK2.all.1_2.bed > HK2.all.1_2.heat.bed
awk 'BEGIN{OFS="\t"}{if (NR>1){print $1,$2,$3,"none",".","+"}}' HK2.all.1_5.bed > HK2.all.1_5.heat.bed


In [29]:
!ls

HK2.all.1_2.bed			      HK2_T2_WT.1_2.bed
HK2.all.1_2.heat.bed		      HK2_T2_WT.1_5.bed
HK2.all.1_5.bed			      HK2_T2_WT.merge.counts.anno.pval.txt
HK2.all.1_5.heat.bed		      HK2_T3_WT.1_2.bed
HK2_T1_WT.1_2.bed		      HK2_T3_WT.1_5.bed
HK2_T1_WT.1_5.bed		      HK2_T3_WT.merge.counts.anno.pval.txt
HK2_T1_WT.merge.counts.anno.pval.txt


bin size = 100

In [34]:
bwpath = '/mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/'
# exp = 'HK2'
exp = '293'
bwfiles = glob.glob(bwpath+exp+'*.bw')
bwfiles.sort()
bwfiles = bwfiles[:5]
for name in names:
    outBed = f'{exp}.all.{name}.heat.bed'
    line = 'computeMatrix reference-point --referencePoint center -S {} -R {} --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 100 --skipZeros --missingDataAsZero -o {}.Ts.ENL.peak.gz -p 8'.format(' '.join(bwfiles), outBed, '{}'.format(name))
    print(line)
    os.system(line)

computeMatrix reference-point --referencePoint center -S /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_ENL_treat.nsp.bw /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T1_treat.nsp.bw /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T2_treat.nsp.bw /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T3_treat.nsp.bw /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_Y78A_treat.nsp.bw -R HK2.all.1_2.heat.bed --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 100 --skipZeros --missingDataAsZero -o 1_2.Ts.ENL.peak.gz -p 8
computeMatrix reference-point --referencePoint center -S /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_ENL_treat.nsp.bw /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T1_treat.nsp.bw /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T2_treat.nsp.bw /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T3_treat.nsp.bw /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_Y78A_treat.nsp.bw -R HK2.all.1_5.heat.bed --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSi

In [38]:
# plot the heatmap
for name in names:
    line = 'plotHeatmap --sortUsingSamples 1 -m {}.Ts.ENL.peak.gz -out HK2.{}.peak.pdf --outFileNameMatrix {} --colorMap Blues Blues Blues Blues Blues --dpi 300 --heatmapHeight 10 --refPointLabel center'.format(name, name, name+'.heatmap.peak.mat')
    print(line)
    os.system(line)

plotHeatmap --sortUsingSamples 1 -m 1_2.Ts.ENL.peak.gz -out HK2.1_2.peak.pdf --outFileNameMatrix 1_2.heatmap.peak.mat --colorMap Blues Blues Blues Blues Blues --dpi 300 --heatmapHeight 10 --refPointLabel center
plotHeatmap --sortUsingSamples 1 -m 1_5.Ts.ENL.peak.gz -out HK2.1_5.peak.pdf --outFileNameMatrix 1_5.heatmap.peak.mat --colorMap Blues Blues Blues Blues Blues --dpi 300 --heatmapHeight 10 --refPointLabel center


## replot the heatmap with shorter height

In [40]:
line = "plotHeatmap --sortUsingSamples 1 -m 1_5.Ts.ENL.peak.gz -out HK2.1_5.peak.pdf --outFileNameMatrix 1_5.heatmap.peak.mat --colorMap Blues Blues Blues Blues Blues --dpi 300 --heatmapHeight 5 --refPointLabel center"
print(line)
os.system(line)

0

In [ ]:
! wc -l 

## Generate the table for the peak heatmap using peak center

In [6]:
def convert_deeptools_gz_to_df(file_name):
    """
    file_name: the deeptools computeMatrix output matrix gz file
    caculate the avg signal per sample across all the bins
    """
    with gzip.open(file_name, 'rt', encoding='utf-8') as f:
        lines = f.readlines()
    line = lines[0]
    a = line.split('@')[-1].split('\\')[0]
    a = re.sub('true', 'True', a)
    a = re.sub('false', 'False', a)
    a = re.sub('null', 'False', a)    
    info = eval(a)
    with open(f'{file_name}.txt', 'w') as f:
        f.writelines(lines[1:])

    df = pd.read_csv(f'{file_name}.txt', sep='\t', header=None) 
    for i, sample in enumerate(info['sample_labels']):
        start = info['sample_boundaries'][i] + 6
        end = info['sample_boundaries'][i+1] + 6    
        df[sample] = df.iloc[:, start:end].mean(axis=1)

    df.sort_values(by=[info['sample_labels'][0]], inplace=True, ascending=False)     
    return df

In [49]:
!ls

1_2.Ts.ENL.peak.gz	HK21_2.peak.sig.txt
1_2.Ts.ENL.peak.gz.txt	HK21_5.peak.sig.txt
1_2.heatmap.peak.mat	HK2_T1_WT.1_2.bed
1_5.Ts.ENL.peak.gz	HK2_T1_WT.1_5.bed
1_5.Ts.ENL.peak.gz.txt	HK2_T1_WT.merge.counts.anno.pval.txt
1_5.heatmap.peak.mat	HK2_T2_WT.1_2.bed
HK2.1_2.peak.pdf	HK2_T2_WT.1_5.bed
HK2.1_5.peak.pdf	HK2_T2_WT.merge.counts.anno.pval.txt
HK2.all.1_2.bed		HK2_T3_WT.1_2.bed
HK2.all.1_2.heat.bed	HK2_T3_WT.1_5.bed
HK2.all.1_5.bed		HK2_T3_WT.merge.counts.anno.pval.txt
HK2.all.1_5.heat.bed


In [48]:
for name in names:
    df = convert_deeptools_gz_to_df(file_name = f'{name}.Ts.ENL.peak.gz')
    df.iloc[:, [0,1,2,3,4,5,-5,-4,-3,-2,-1]].to_csv(f'{exp}{name}.peak.sig.txt', index=False, sep='\t') 

## add the delta heatmap for the above sampels
Just heard back from Liling, actually she wants to change the delta heatmap color theme back to grey/black and orange that we used before. Could you please make this change for all the delta heatmap? In the FC >1.2 and FC>1.5 delta heatmap, right now we only have signals for wt, T1, T2 and T3, please add T1-wt, T2-wt and T3-wt. Attached is an old version of this figure for your reference, keep the blue color for individual sample, use the same organge-black/grey for delta map, but no need to label gene names on the right. **2018-8-3**

In [43]:
!ls *.gz
!ls *.pdf

1_2.Ts.ENL.peak.gz  1_5.Ts.ENL.peak.gz
HK2.1_2.peak.pdf  HK2.1_5.peak.pdf


## Generate the table for the peak region

In [7]:
# function for formating the chr start end into bed format
# if the last column has sign ('+' or '-'), then prepare to swap tss and tes for '-'
def convert_region_to_bed(bedfile, outname, header=0):
    df = pd.read_csv(bedfile,sep='\t',header=header)
    df.columns = np.arange(0,df.shape[1])
    # detect the strand and swap the start end when negative strand
    # the last column should be strand
    if df.iloc[0,-1] == '-' or df.iloc[0,-1] == '+':
        idx = df.iloc[:,-1] == '-'
        # swap the tss tes according to the strand
        df.loc[idx,[1,2]] = df.loc[idx,[2,1]].values
        # modify the strand
        df.iloc[:,-1] = '+'
        
    df.loc[:,3] = np.arange(df.shape[0])# for gene name column
    df.loc[:,4] = 0 # for bed format (value)
    df.loc[:,5] = '+'
    df = df.reindex(range(6), axis='columns')
    print('writing the output --> {}'.format(outname))
    print(f'{outname} --->: reformated bed_file as the input of the bigWigAverageoverBed')
    df.to_csv(outname,sep='\t',index=False,header=None)
    
    
def comp_avg_sig_from_bw_by_bed(bedfile, bw_files, which_column=2):
    """
    bedfile: the input merged bedfile which needs to extract the signal from
    bw_files: list of bw files whose signal is extracted from
    which_column: 3 is the acutal counts column, 4 is the acutal avg signal column;
    but, we need to use the first column as index, then 2 is the counts column;
    outname: reformat the bedfile to the normal 6 columns bed file
    will save the output file in the same folder as the bedfile
    """
    outname = bedfile.split('.bed')[0] + '.str.bed'
    # convert the 3 columns bed to the normal bed file
    convert_region_to_bed(bedfile, outname)
    for i,bw_file in enumerate(bw_files):
        command = line = f'~/xc3/software/bigWigAverageOverBed {bw_file} {outname} avg_sig_{i}.txt'
        print('execute --> {}'.format(command))
        os.system(command)
    dfs = []
    for i in range(len(bw_files)):
        dfs.append(pd.read_csv(f'avg_sig_{i}.txt',sep='\t',header=None, index_col=0))
    df = pd.DataFrame([dfs[0].iloc[:,0]] + # iloc[:,0]: the width
                      [dfi.iloc[:,which_column] for dfi in dfs]).T # conbine the columns together
    df.columns = ['width'] + [bw_file.split('/')[-1].split('.')[0] for bw_file in bw_files]
    df_bed = pd.read_csv(bedfile, sep='\t', header=None, names = ['chr', 'start', 'end'])
    df_out = df_bed.join(df)
    print(f"output file -->: {bedfile.split('.bed')[0] + '.counts.txt'}")
    df_out.to_csv(bedfile.split('.bed')[0] + '.counts.txt', sep='\t', index=False)

In [83]:
exp = 'HK2'
bwfiles = glob.glob(bwpath+exp+'*.bw')
bwfiles.sort()
bwfiles = bwfiles[:5]
for name in names:
    bed_file = f'{exp}.all.{name}.bed'
    print('generate the signal for',bed_file)
    comp_avg_sig_from_bw_by_bed(bed_file, bwfiles, 3)
    

generate the signal for HK2.all.1_2.bed
writing the output --> HK2.all.1_2.str.bed
HK2.all.1_2.str.bed --->: reformated bed_file as the input of the bigWigAverageoverBed
execute --> ~/xc3/software/bigWigAverageOverBed /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_ENL_treat.nsp.bw HK2.all.1_2.str.bed avg_sig_0.txt
execute --> ~/xc3/software/bigWigAverageOverBed /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T1_treat.nsp.bw HK2.all.1_2.str.bed avg_sig_1.txt
execute --> ~/xc3/software/bigWigAverageOverBed /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T2_treat.nsp.bw HK2.all.1_2.str.bed avg_sig_2.txt
execute --> ~/xc3/software/bigWigAverageOverBed /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T3_treat.nsp.bw HK2.all.1_2.str.bed avg_sig_3.txt
execute --> ~/xc3/software/bigWigAverageOverBed /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_Y78A_treat.nsp.bw HK2.all.1_2.str.bed avg_sig_4.txt
output file -->: HK2.all.1_2.counts.txt
generate the signal for HK2.all.1_5.bed
writing the output -->

### re-rank the file accordi ng to the heatmap

In [139]:
def combine_row(row):
    row = [str(ele) for ele in row]
    return '|'.join(row)

for name in names:
    df1 = pd.read_csv(f'{name}.peak.sig.txt', sep='\t')
    df1.index = df1.iloc[:,:3].apply(combine_row, axis=1)
    df2 = pd.read_csv(f'{exp}.all.{name}.counts.txt', sep='\t')
    df2.index = df2.iloc[:,:3].apply(combine_row, axis=1)
    df2 = df2.reindex(df1.index)
    df2.to_csv(f'{exp}.all.{name}.sig.txt', sep='\t', index=False)


In [84]:
def annoate_bed_to_gene(infile, outfile, cutoff=5000):
    ref, tss_set = {}, set()
    reffile = '/mount/weili3/xc3/genomes/hg19.refGene.txt'
    for line in open(reffile):
        col = line.split('\t')
        name, cr, strand, TSS, TES, symbol = col[1], col[2], col[3], int(col[4]), int(col[5]), col[12]
        if strand == '-': TSS, TES = TES, TSS
        if cr not in ref: ref[cr] = []
        if (cr,TSS,strand) not in tss_set:
            ref[cr].append((name,symbol,strand,TSS,TES))
            tss_set.add((cr,TSS,strand))
    for cr in ref: ref[cr].append(('none','none','none',0,0))
    
    # annotate the file
    text = open(infile).readlines()
    fout = open(outfile, 'w')
    print('processing on {}\n will output {}\n'.format(infile,outfile))
    for line in text:
        col = line.split('\t')
        try: cr, start, end = col[0], int(col[1]), int(col[2])
        except: 
            fout.write(line[:-1]+'\twithin_genebody\tnearest_TSS\tdistance\n')
            continue
        if cr not in ref: continue
        peak = (start + end) / 2
        genes, genebody, genes0, genebody0 = [], [], [], []
        for name, symbol, strand, TSS, TES in ref[cr]:
            if strand == '+':
                dist = end - TSS
                if abs(start-TSS)<abs(dist):dist=start-TSS

            elif strand == '-':
                dist = TSS - end
                if abs(TSS - start)<abs(dist):dist=TSS - start

            elif strand != 'none': raise ValueError
            if abs(dist) <= cutoff: genes.append((abs(dist),symbol,dist)) 
            if (start - TSS) * (start - TES) <= 0: genebody.append((abs(dist),symbol,dist))
            elif (end - TSS) * (end - TES) <= 0: genebody.append((abs(dist),symbol,dist))
            elif (start - TSS) * (end - TES) <=0: genebody.append((abs(dist),symbol,dist))
        genes, genebody = sorted(genes), sorted(genebody)
        for g in sorted(genes):
            if g[1] not in [x[1] for x in genes0]: genes0.append(g)
        for g in sorted(genebody):
            if g[1] not in [x[1] for x in genebody0]: genebody0.append(g)
        if any(genes0):
            symbols = ','.join([x[1] for x in genes0])
            dists = ','.join(['%d' % x[2] for x in sorted(genes0)])
        else: symbols, dists = 'none', 'none'
        if any(genebody): body = ','.join([x[1] for x in genebody0])
        else: body = 'none'
        fout.write(line[:-1] + '\t%s\t%s\t%s\n' % (body,symbols,dists))
    fout.close()   

In [86]:
!ls

1_2.Ts.ENL.peak.gz	HK2.all.1_5.bed
1_2.Ts.ENL.peak.gz.txt	HK2.all.1_5.counts.txt
1_2.heatmap.peak.mat	HK2.all.1_5.heat.bed
1_5.Ts.ENL.peak.gz	HK2.all.1_5.str.bed
1_5.Ts.ENL.peak.gz.txt	HK2_T1_WT.1_2.bed
1_5.heatmap.peak.mat	HK2_T1_WT.1_5.bed
HK2.1_2.peak.pdf	HK2_T1_WT.merge.counts.anno.pval.txt
HK2.1_2.peak.sig.txt	HK2_T2_WT.1_2.bed
HK2.1_5.peak.pdf	HK2_T2_WT.1_5.bed
HK2.1_5.peak.sig.txt	HK2_T2_WT.merge.counts.anno.pval.txt
HK2.all.1_2.bed		HK2_T3_WT.1_2.bed
HK2.all.1_2.counts.txt	HK2_T3_WT.1_5.bed
HK2.all.1_2.heat.bed	HK2_T3_WT.merge.counts.anno.pval.txt
HK2.all.1_2.str.bed


In [88]:
for name in names:
    annoate_bed_to_gene(f'{exp}.{name}.peak.sig.txt', f'{exp}.all.{name}.sig.anno.txt')

processing on HK2.1_2.peak.sig.txt
 will output HK2.all.1_2.sig.anno.txt

processing on HK2.1_5.peak.sig.txt
 will output HK2.all.1_5.sig.anno.txt



# Generate the coup delta heatmap

In [43]:
#format the bed file into the bed file add three columns more; 
names = ['1_2','1_5']
for name in names:
    Bed = f'{exp}.all.{name}.sig.anno.txt'
    outBed = f'{exp}.all.{name}.new.heat.bed'
    line = "awk 'BEGIN{OFS=\"\\t\"}{if (NR>0){print $1,$2,$3,NR-2,\".\",\"+\"}}'" + f' {Bed} > {outBed}'
    print(line)
    os.system(line)

awk 'BEGIN{OFS="\t"}{if (NR>1){print $1,$2,$3,NR-2,".","+"}}' 293.all.1_2.sig.anno.txt > 293.all.1_2.new.heat.bed
awk 'BEGIN{OFS="\t"}{if (NR>1){print $1,$2,$3,NR-2,".","+"}}' 293.all.1_5.sig.anno.txt > 293.all.1_5.new.heat.bed


In [10]:
ctrl = 'ENL'
treats = ['T1', 'T2', 'T3', 'Y78A']

In [101]:
samples = treats + [ctrl]
for sample in samples:
    bwfile = glob.glob(bwpath + exp + '*' + sample + '*.bw')[0]
    outBed = f"{exp}.all.1_2.new.heat.bed"
    print(bwfile)
    line = f'computeMatrix reference-point --referencePoint center -S {bwfile} -R {outBed} --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 100 --skipZeros --missingDataAsZero -o PeakMatrix_{exp}_F_{sample}_co-UP_genes.gz -p 8'
    print(line + '\n****\n')
    os.system(line)

/mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T1_treat.nsp.bw
computeMatrix reference-point --referencePoint center -S /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T1_treat.nsp.bw -R HK2.all.1_2.new.heat.bed --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 100 --skipZeros --missingDataAsZero -o PeakMatrix_HK2_F_T1_co-UP_genes.gz -p 8
****

/mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T2_treat.nsp.bw
computeMatrix reference-point --referencePoint center -S /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T2_treat.nsp.bw -R HK2.all.1_2.new.heat.bed --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 100 --skipZeros --missingDataAsZero -o PeakMatrix_HK2_F_T2_co-UP_genes.gz -p 8
****

/mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T3_treat.nsp.bw
computeMatrix reference-point --referencePoint center -S /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/HK2_F_T3_treat.nsp.bw -R HK2.all.1_2.new.heat.bed --beforeRegionStartLength 5000 --afterRegionStartLength

In [42]:
!ls -alt
!head 293.all.1_5.new_heat.bed

total 8564
drwxr-xr-x  2 jlyu lilab    4096 Oct  3 23:45 .
-rw-r--r--  1 jlyu lilab    1653 Oct  3 23:41 293.all.1_5.new.heat.bed
-rw-r--r--  1 jlyu lilab    8079 Oct  3 23:41 293.all.1_2.new.heat.bed
-rw-r--r--  1 jlyu lilab   25044 Oct  3 23:41 293.all.1_2.sig.anno.txt
-rw-r--r--  1 jlyu lilab    5691 Oct  3 23:41 293.all.1_5.sig.anno.txt
drwxr-xr-x 11 jlyu lilab    4096 Oct  3 22:51 ..
-rw-r--r--  1 jlyu lilab   15726 Oct  3 22:15 PeakMatrix_HK2_F_ENL_co-UP_genes.1_5.gz
-rw-r--r--  1 jlyu lilab   14997 Oct  3 22:15 PeakMatrix_HK2_F_Y78A_co-UP_genes.1_5.gz
-rw-r--r--  1 jlyu lilab   17514 Oct  3 22:15 PeakMatrix_HK2_F_T3_co-UP_genes.1_5.gz
-rw-r--r--  1 jlyu lilab   16699 Oct  3 22:15 PeakMatrix_HK2_F_T2_co-UP_genes.1_5.gz
-rw-r--r--  1 jlyu lilab   17019 Oct  3 22:15 PeakMatrix_HK2_F_T1_co-UP_genes.1_5.gz
-rw-r--r--  1 jlyu lilab  616084 Aug 28 00:10 TsWT.sorted.bed
-rw-r--r--  1 jlyu lilab   52286 Aug 18 03:38 PeakMatrix_HK2_F_ENL_co-UP_genes.gz
-rw-r--r--  1 jlyu lilab   50352 Aug

In [51]:
exp = '293'
samples = treats + [ctrl]
for sample in samples:
    bwfile = glob.glob(bwpath + exp + '*' + sample + '*.bw')[0]
    outBed = f"{exp}.all.1_5.new.heat.bed"
    print(bwfile)
    line = f'computeMatrix reference-point --referencePoint center -S {bwfile} -R {outBed} --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 100 --skipZeros --missingDataAsZero -o PeakMatrix_{exp}_F_{sample}_co-UP_genes.1_5.gz -p 8'
    print(line + '\n****\n')
    os.system(line)

/mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/293_F_T1_treat.nsp.bw
computeMatrix reference-point --referencePoint center -S /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/293_F_T1_treat.nsp.bw -R 293.all.1_5.new.heat.bed --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 100 --skipZeros --missingDataAsZero -o PeakMatrix_293_F_T1_co-UP_genes.1_5.gz -p 8
****

/mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/293_F_T2_treat.nsp.bw
computeMatrix reference-point --referencePoint center -S /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/293_F_T2_treat.nsp.bw -R 293.all.1_5.new.heat.bed --beforeRegionStartLength 5000 --afterRegionStartLength 5000 --binSize 100 --skipZeros --missingDataAsZero -o PeakMatrix_293_F_T2_co-UP_genes.1_5.gz -p 8
****

/mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/293_F_T3_treat.nsp.bw
computeMatrix reference-point --referencePoint center -S /mount/weili2/lilab/xc3/ENL2/ChIP/bw_nsp/293_F_T3_treat.nsp.bw -R 293.all.1_5.new.heat.bed --beforeRegionStartLength 5000 --afterRegionSta

In [53]:
def generate_outer_links(files):
    public_url_pre = "http://dldcc-web.brc.bcm.edu/lilab/xc3/ENL2/ChIP/tmp/"
    public_url_folder = "/mount/weili2/lilab/xc3/ENL2/ChIP/tmp/"
    for ifile in files:
        file_name = ifile.split('/')[-1]
        line = f"cp {ifile} {public_url_folder}{ifile}"
        print(line)
        os.system(line)
        print(f'{public_url_pre}{ifile}')

In [55]:
public_url_pre = "http://dldcc-web.brc.bcm.edu/lilab/xc3/ENL2/ChIP/tmp/"
public_url_folder = "/mount/weili2/lilab/xc3/ENL2/ChIP/tmp/"
work_path = "/home/jlyu/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/Run_On_Lvjie/4.coup_peaks/"
#files = glob.glob(work_path + '*293*1_5.gz')
files = glob.glob('*293*1_5.gz')
print(files)

['PeakMatrix_293_F_T1_co-UP_genes.1_5.gz', 'PeakMatrix_293_F_T2_co-UP_genes.1_5.gz', 'PeakMatrix_293_F_T3_co-UP_genes.1_5.gz', 'PeakMatrix_293_F_Y78A_co-UP_genes.1_5.gz', 'PeakMatrix_293_F_ENL_co-UP_genes.1_5.gz']


In [57]:
for ifile in files:
    source = work_path + ifile
    symbol = public_url_folder + ifile
    line = f"cp {source} {public_url_folder}{ifile}"
    print(line)
    os.system(line)
    print(f'{public_url_pre}{ifile}')
    
print(len(files))

cp /home/jlyu/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/Run_On_Lvjie/4.coup_peaks/PeakMatrix_293_F_T1_co-UP_genes.1_5.gz /mount/weili2/lilab/xc3/ENL2/ChIP/tmp/PeakMatrix_293_F_T1_co-UP_genes.1_5.gz
http://dldcc-web.brc.bcm.edu/lilab/xc3/ENL2/ChIP/tmp/PeakMatrix_293_F_T1_co-UP_genes.1_5.gz
cp /home/jlyu/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/Run_On_Lvjie/4.coup_peaks/PeakMatrix_293_F_T2_co-UP_genes.1_5.gz /mount/weili2/lilab/xc3/ENL2/ChIP/tmp/PeakMatrix_293_F_T2_co-UP_genes.1_5.gz
http://dldcc-web.brc.bcm.edu/lilab/xc3/ENL2/ChIP/tmp/PeakMatrix_293_F_T2_co-UP_genes.1_5.gz
cp /home/jlyu/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/Run_On_Lvjie/4.coup_peaks/PeakMatrix_293_F_T3_co-UP_genes.1_5.gz /mount/weili2/lilab/xc3/ENL2/ChIP/tmp/PeakMatrix_293_F_T3_co-UP_genes.1_5.gz
http://dldcc-web.brc.bcm.edu/lilab/xc3/ENL2/ChIP/tmp/PeakMatrix_293_F_T3_co-UP_genes.1_5.gz
cp /home/jlyu/xc3/experiment

# 4 Motif analysis

1） 333 genes with FC>1.2 enhanced binding; 2）87 genes with FC>1.5; 3) T3 vs.WT FC>1.5 enhanced binding

In [106]:
os.chdir('/home/jlyu/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/Run_On_Lvjie/')

In [110]:
workPath = '/home/jlyu/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/Run_On_Lvjie/6.res_motif/'
os.chdir(workPath)


# 4.1 extract the sequence for the three gene lists

In [266]:
# read the gene's tss and tes and save to the bed file
geneLists = ['T1_2_3 co-UP FC1.2.txt', 'T1_2_3 co-UP FC1.5.txt', '293_T3_WT.merge.counts.anno.genes1_5.txt']
names = ['coup.1_2', 'coup.1_5', 'T3WT.1_5']
gene_list = geneLists[0]
for i, gene_list in enumerate(geneLists):
    with open(gene_list, 'r') as f:
        genes = f.readlines()
        genes = [gene.rstrip() for gene in genes]
        df = df_genes.reindex(genes)
        df.loc[:,'strand'] = '+'
        print(f'process on {gene_list}')
        df['tes'] = df['tss'] + 2000
        df['tss'] = df['tss'] - 2000
        df.to_csv(f'{names[i]}.motif.bed', sep='\t', header=None, index=False)

process on T1_2_3 co-UP FC1.2.txt
process on T1_2_3 co-UP FC1.5.txt
process on 293_T3_WT.merge.counts.anno.genes1_5.txt


In [111]:
!ls

HK2.all.1_2.new.heat.bed  HK2.all.1_2.sig.anno.txt  HK2.all.1_5.new.heat.bed


In [117]:
# extract the sequence for the corrdinates
names = ['1_2', '1_5']
for name in names:
    bedFile = f'HK2.all.{name}.new.heat.bed'
    outTxt1 = f'{name}.motif.tmp.txt'
    line = "awk 'BEGIN{OFS=\"\\t\"}{if (NR>0) {mid = int( ($2+$3) / 2); print $1, mid-2000, mid+2000, $4, $5, $6;}}\' " + bedFile + f"> {outTxt1}"
    print(line)
    os.system(line)
    outTxt = f'{name}.motif.txt'
    line = f"bedtools getfasta -fi /mount/weili3/xc3/genomes/hg19_trim.fa -bed {outTxt1} -fo {outTxt}"
    print(line)
    os.system(line)
    line = "awk \'!a[$0]++\'" + f" {outTxt} > {name}.motif.uniq.txt"
    print(line)
    os.system(line)

awk 'BEGIN{OFS="\t"}{if (NR>0) {mid = int( ($2+$3) / 2); print $1, mid-2000, mid+2000, $4, $5, $6;}}' HK2.all.1_2.new.heat.bed> 1_2.motif.tmp.txt
bedtools getfasta -fi /mount/weili3/xc3/genomes/hg19_trim.fa -bed 1_2.motif.tmp.txt -fo 1_2.motif.txt
awk '!a[$0]++' 1_2.motif.txt > 1_2.motif.uniq.txt
awk 'BEGIN{OFS="\t"}{if (NR>0) {mid = int( ($2+$3) / 2); print $1, mid-2000, mid+2000, $4, $5, $6;}}' HK2.all.1_5.new.heat.bed> 1_5.motif.tmp.txt
bedtools getfasta -fi /mount/weili3/xc3/genomes/hg19_trim.fa -bed 1_5.motif.tmp.txt -fo 1_5.motif.txt
awk '!a[$0]++' 1_5.motif.txt > 1_5.motif.uniq.txt


In [118]:
!ls

1_2.motif.tmp.txt   1_5.motif.tmp.txt	HK2.all.1_2.new.heat.bed
1_2.motif.txt	    1_5.motif.txt	HK2.all.1_2.sig.anno.txt
1_2.motif.uniq.txt  1_5.motif.uniq.txt	HK2.all.1_5.new.heat.bed


## 4.2 look for enriched motifs in wild type ENL binding peaks.

In [12]:
!head /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/293_F_ENL.nsp_peaks.bed
!wc -l /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/293_F_ENL.nsp_peaks.bed
!cp /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/293_F_ENL.nsp_peaks.bed  ./
!mv 293_F_ENL.nsp_peaks.bed ENL.motif.bed

chr1	858702	861593	MACS_peak_1	1093.75
chr1	870327	871455	MACS_peak_2	97.85
chr1	875392	878559	MACS_peak_3	676.52
chr1	932422	936620	MACS_peak_4	1067.60
chr1	955250	956213	MACS_peak_5	86.76
chr1	1709153	1710245	MACS_peak_6	94.82
chr1	1821251	1822759	MACS_peak_7	346.39
chr1	2159906	2163136	MACS_peak_8	1053.44
chr1	2245753	2246888	MACS_peak_9	88.14
chr1	3568419	3569644	MACS_peak_10	98.29
3298 /mount/weili3/xc3/ENL2_ChIP/res_nsp_lambda/293_F_ENL.nsp_peaks.bed


In [13]:
name = 'ENL'
bedFile = f'{name}.motif.bed'
outTxt = f'{name}.motif.txt'
line = f"bedtools getfasta -fi /mount/weili3/xc3/genomes/hg19_trim.fa -bed {bedFile} -fo {outTxt}"
print(line)
os.system(line)
line = "awk \'!a[$0]++\'" + f" {outTxt} > {name}.motif.uniq.txt"
print(line)
os.system(line)

bedtools getfasta -fi /mount/weili3/xc3/genomes/hg19_trim.fa -bed ENL.motif.bed -fo ENL.motif.txt
awk '!a[$0]++' ENL.motif.txt > ENL.motif.uniq.txt


0

In [141]:
!pwd
os.chdir('/home/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/res_heatmap_coup_peaks')

/home/xc3/experiment/ENL2/Chip-seq_analysis/no_spike_in/re_do_mapping_with_lambda/res_heatmap_coup_peaks


## 5. Test all the pre-binding peaks signal

In [143]:
bwfiles

['/home/xc3/output/ENL2/ChIP/bw_nsp/293_F_ENL_treat.nsp.bw',
 '/home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T1_treat.nsp.bw',
 '/home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T2_treat.nsp.bw',
 '/home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T3_treat.nsp.bw',
 '/home/xc3/output/ENL2/ChIP/bw_nsp/293_F_Y78A_treat.nsp.bw']

In [149]:
treats = ['T1','T2','T3']
treat = treats[0]
ctrl = 'ENL'
[glob.glob(bwpath+exp+'*'+treat+'*.bw')[0], glob.glob(bwpath+exp+'*'+ctrl+'*.bw')[0]]
bed_file = f'{exp}_{treat}_WT.1_2.bed'
!head $bed_file

chr1	8482987	8484389
chr1	15943726	15944596
chr1	16173761	16176666
chr1	26734734	26735700
chr1	27286358	27287134
chr1	38229600	38230601
chr1	57043618	57045418
chr1	61547212	61550060
chr1	65431315	65432543
chr1	78097423	78099183


In [171]:
bwpath = '/home/xc3/output/ENL2/ChIP/bw_nsp/'
exp = '293'
bwfiles.sort()
for treat in treats:
    bed_file = f'{exp}_{treat}_WT.1_2.bed'
    bwfiles = [glob.glob(bwpath+exp+'*'+treat+'*.bw')[0],
          glob.glob(bwpath+exp+'*'+ctrl+'*.bw')[0]]
    print('generate the signal for',bed_file)
    comp_avg_sig_from_bw_by_bed(bed_file, bwfiles, 3)
    print('********'+'\n')

generate the signal for 293_T1_WT.1_2.bed
writing the output --> 293_T1_WT.1_2.str.bed
293_T1_WT.1_2.str.bed --->: reformated bed_file as the input of the bigWigAverageoverBed
execute --> ~/software/bigWigAverageOverBed /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T1_treat.nsp.bw 293_T1_WT.1_2.str.bed avg_sig_0.txt
execute --> ~/software/bigWigAverageOverBed /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_ENL_treat.nsp.bw 293_T1_WT.1_2.str.bed avg_sig_1.txt
output file -->: 293_T1_WT.1_2.counts.txt
********
generate the signal for 293_T2_WT.1_2.bed
writing the output --> 293_T2_WT.1_2.str.bed
293_T2_WT.1_2.str.bed --->: reformated bed_file as the input of the bigWigAverageoverBed
execute --> ~/software/bigWigAverageOverBed /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_T2_treat.nsp.bw 293_T2_WT.1_2.str.bed avg_sig_0.txt
execute --> ~/software/bigWigAverageOverBed /home/xc3/output/ENL2/ChIP/bw_nsp/293_F_ENL_treat.nsp.bw 293_T2_WT.1_2.str.bed avg_sig_1.txt
output file -->: 293_T2_WT.1_2.counts.txt
********


In [185]:
# df = pd.read_csv('1_2.peak.sig.txt', sep='\t')
df = pd.read_csv('293_T1_WT.1_2.counts.txt', sep='\t')

In [186]:
df['fc'] = df.iloc[:,-2].div(df.iloc[:,-1])

In [187]:
df[df['fc'] < 1.2]

,chr,start,end,width,293_F_T1_treat,293_F_ENL_treat,fc


In [12]:
bwpath = '/home/xc3/output/ENL2/ChIP/bw_nsp/'